In [ ]:
from lec_utils import *

<div class="alert alert-info" markdown="1">

#### Lecture 9

# Web Scraping

### EECS 398-003: Practical Data Science, Fall 2024

<small><a style="text-decoration: none" href="https://practicaldsc.org">practicaldsc.org</a> • <a style="text-decoration: none" href="https://github.com/practicaldsc/fa24">github.com/practicaldsc/fa24</a></small>
    
</div>

### Announcements 📣

- Homework 4 is due on **Thursday**.
- Homework 2 scores are available on Gradescope.
- The Midterm Exam is in a few weeks – start working through old exam problems [**here**](https://study.practicaldsc.org/).<br><small>You can even see an example old exam PDF.</small>

### Agenda

- Recap: Handling missing values.
- Introduction to HTTP.
- The structure of HTML.
- Parsing HTML.
    - Example: Scraping quotes.
    - Example: Scraping the Happening @ Michigan page.

## Recap: Handling missing values

---

### Summary of imputation techniques

- Consider whether values are missing intentionally, or whether there's a default replacement.

- Listwise deletion.<br><small>Drop, or ignore, missing values.</small>

- (Conditional) mean imputation.<br><small>Fill in missing values with the mean of observed values. If there's a reason to believe the missingness depends on another categorical column, fill in missing values with the observed mean separately for each category.</small>

- (Conditional) Probabilistic imputation.<br><small>Fill in missing values with a random sample of observed values. If there's a reason to believe the missingness depends on another categorical column, fill in missing values with a random sample drawn separately for each category.</small>

- Regression imputation.<br><small>Predict missing values using other features.</small>

<div class="alert alert-success">
<h3>Activity</h3>

Work on [**UCSD DSC 80 Spring 2022 Final Exam, Problem 5 (Parts 1-4)**](https://practice.dsc80.com/sp22-final/index.html#Problem-5).

### Missingness mechanisms

There are three key **missingness mechanisms**, which describe _how_ data in a column can be missing.

- **Missing completely at random (MCAR)**: Data are MCAR if the chance that a value is missing is **completely independent** of other columns and the actual missing value.<br><small>Example: Suppose that after the Midterm Exam, I randomly choose 5 scores to delete on Gradescope, meaning that 5 students have missing grades. MCAR is **ideal, but rare!**</small>

- **Missing at random (MAR)**: Data are MAR if the chance that a value is missing **depends on other columns**.<br><small>Example: Suppose that after the Midterm Exam, I randomly choose 5 scores to delete on Gradescope **among** sophomore students. Now, scores are missing at random **dependent on class standing**.</small>

- **Not missing at random (NMAR)**: Data are NMAR if the chance that a value is missing **depends on the actual missing value itself**.<br><small>Example: Suppose that after the Midterm Exam, I randomly delete 5 of the 10 lowest scores on Gradescope. Now, scores are **not** missing at random, since the chance a value is missing depends on how large it is.</small>

- Statistical imputation packages usually assume data are MAR.<br><small>MCAR is usually unrealistic to assume. If data are NMAR, you can't impute missing values, since the other features in your data **can't explain** the missingness.</small>

### How do we know if data are MCAR?

- It seems that if our data are MCAR, there is no risk to dropping missing values.<br><small>In the MCAR setting, just imagine we're being given a large, random sample of the true dataset.</small>

- If the data are not MCAR, though, then dropping the missing values will introduce bias.<br><small>For instance, suppose we asked people "How much do you give to charity?" People who give little are less likely to respond, so the average response is **biased high**.</small>

- There is no perfect procedure for determining if our data are MCAR, MAR, or NMAR; we mostly have to use our understanding of how the data is generated.

- But, we can _try_ to determine whether $Y_\text{missing}$ is similar to $Y$, using the information we **do have** in other columns.<br><small>We did this earlier, when looking at the proportion of missing `'child'` heights for each `'gender'`.</small>

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
<small>Remember that you can always ask questions anonymously at the link above!</small>
    
What lingering questions do we have about how to handle missing values?

## Introduction to HTTP

---

<center><img src="imgs/ds-lifecycle.svg" width="60%"></center>

### Data sources

- Often, the data you need doesn't exist in "clean" `.csv` files.

- **Solution**: Collect your own data from the internet!<br><small>For most questions you can think of, the answer exists somewhere on the internet. If not, you can run our own survey – also on the internet!</small>

### Manual copy-pasting

- If data is already nicely formatted in a table online, sometimes we can easily copy it and paste it into a `.csv` or `.tsv` file.<br><small>`.tsv` stands for "tab-separated values", just like `.csv` stands for "comma-separated values."</small> 

- For example, open the 2024 Michigan Football schedule [**here**](https://mgoblue.com/sports/football/schedule) and click "Text Only".


<center><img src="imgs/mich-schedule.png" width=700><br><small>This is what you should see.</small></center>

- Copy the text in the table at the bottom and save it in a file named `2024-schedule.tsv` in your `data` folder.<br><small>You may need to do some minor reformatting in the `.tsv` file before this works.<br>**As a challenge**, see if you can find a way to do this entirely within your Terminal, i.e. without opening a text editor!</small>

In [ ]:
schedule = pd.read_csv('data/2024-schedule.tsv', sep='\t') 
schedule.head()

- For Wikipedia specifically, you can use [Wikitable2CSV](https://wikitable2csv.ggor.de/), which converts Wikipedia tables to `.csv` files for you.

### Programatically accessing data

- We won't always be able to copy-paste tables from online, and even when we can, it's not easily **reproducible**.<br><small>What if [mgoblue.com](https://mgoblue.com/sports/football/schedule) didn't have a "Text Only" option? Or what if the schedule changes – how can I prevent myself from having to copy-and-paste again?</small>

- There are two ways to programmatically access data on the internet: either **by scraping**, or **through an API**.<br><small>By programmatically, we mean by writing code.</small>

- We will discuss the differences between both approaches, but for now, the important part is that they **both use HTTP**.

- **Note**: We'll cover the essentials of how the internet works for the purposes of accessing data, but for more details, take [EECS 485](https://eecs485.org).

### Goal

- Our goal in today's lecture will be to programatically create a DataFrame containing all of the events at the Happening @ Michigan page, taken from [events.umich.edu](https://events.umich.edu).

- Somehow, we'll start with the link to a webpage that looks like:

<center><img src="imgs/events.png" width=800><small>Today's event page will look different, since I wrote this lecture yesterday!</small></center>

- And end with a DataFrame like this one, all just by writing code:

<table border="1" class="dataframe" width=1200>
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>title</th>
      <th>time</th>
      <th>location</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>International Students Career Series: Coffee Chat with the University Career Center</td>
      <td>Sep 23, 2024 9:00am</td>
      <td>University Career Center, 3200 Student Activities Building, Program Room (3003), 515 E Jefferson St, Ann Arbor, MI, United States</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Alaska Teachers &amp; Personnel Informational Meeting</td>
      <td>Sep 23, 2024 10:00am</td>
      <td></td>
    </tr>
    <tr>
      <th>2</th>
      <td>Michigan in Washington Fall 2024 Application Deadline</td>
      <td>Sep 23, 2024 10:00am</td>
      <td></td>
    </tr>
    <tr>
      <th>3</th>
      <td>EEB Prelim Seminar Series - Evolution of “Collecting” Behavior in Deep Sea Carrier Snails</td>
      <td>Sep 23, 2024 10:30am</td>
      <td>Biological Sciences Building</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Huron Affinity Group Overview (iMatter Teams)</td>
      <td>Sep 23, 2024 11:00am</td>
      <td></td>
    </tr>
  </tbody>
</table>

### The request-response model

- HTTP stands for **Hypertext Transfer Protocol**.<br><small>It was developed in 1989 by Tim Berners-Lee (and friends). The "S" in HTTPS stands for "secure".</small>

- HTTP follows the **request-response** model.

<center><img src='imgs/req-response.png' width=500></center>

- A <b><span style="color:blue">request</span></b> is made by the <b><span style="color:blue">client</span></b>.


- A <b><span style="color:orange">response</span></b> is returned by the <b><span style="color:orange">server</span></b>.


- **Example**: YouTube search 🎥.
    - Consider the following URL: https://www.youtube.com/results?search_query=iphone+16+pro+unboxing.
    - Your web browser, a **client**, makes an HTTP **request** with a search query.
    - The **server**, YouTube, is a computer that is sitting somewhere else.
    - The server returns a **response** that contains the search results.
    - **Note**: ?search_query=iphone+16+pro+unboxing is called a "query string."

### Consequences of the request-response model

- When a request is sent to view content on a webpage, the server must:
    - process your request (i.e. prepare data for the response).
    - send content back to the client in its response.

- Remember, servers are computers.  Someone has to pay to keep these computers running.<br><small>**Every time you access a website, someone has to pay.**</small>

- If you make too many requests, the server may block your IP address, or **you may even take down the website**!<br><small>A journalist scraped and accidentally took down the Cook County Inmate Locater, and as a result, inmate's families weren't able to contact them while the site was down.</small>

### HTTP request methods

- The request methods you will use most often are `GET` and `POST`; see [Mozilla's web docs](https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods) for a detailed list of request methods.    

- `GET` is used to request data **from** a specified resource.<br><small>Almost all of the requests we'll make in this class are `GET` requests.<br>To load websites, your web browser uses a lot of `GET` requests!</small>

- `POST` is used to **send** data to the server. <br><small>For example, uploading a photo to Instagram or entering credit card information on Amazon.</small>

- You can make requests directly in your Terminal using the `curl` method, which you'll learn more about in EECS 485. **Here, we'll make requests using the `requests` Python module!**<br><small>There are other packages that work similarly (e.g. `urllib`), but `requests` is arguably the easiest to use.</small>

In [ ]:
import requests

### Example: `GET` requests via `requests`

- For instance, let's access the source code of the events page, https://events.umich.edu.

In [ ]:
res = requests.get('https://events.umich.edu') 

- `res` is now a `Response` object.

In [ ]:
res

- The `text` attribute of `res` is a string that containing the entire response.

In [ ]:
type(res.text) 

In [ ]:
len(res.text) 

In [ ]:
print(res.text[:2000]) 

- The response is a string containing **HTML**, the markup language used to format information on the internet. The events data we're looking for is in `res.text` _somewhere_, but we have to search for it and extract it.

<div class="alert alert-danger" markdown="1">

#### Reference Slide

### Example: `POST` requests via `requests`

- The following call to `requests.post` makes a post request to https://httpbin.org/post, with a `'name'` parameter of `'Go Blue'`.

In [ ]:
post_res = requests.post('https://httpbin.org/post',
                         data={'name': 'Go Blue'})
post_res

In [ ]:
post_res.text

- Now, the response is a string describing a JSON object. We'll learn how to work with these later in the lecture and into Lecture 10, but for now, note that we can use the `.json()` method to convert it to a Python dictionary.

In [ ]:
post_res.json()

- What happens when we try and make a `POST` request somewhere where we're unable to?

In [ ]:
yt_res = requests.post('https://youtube.com',
                       data={'name': 'Go Blue'})
yt_res

In [ ]:
# This takes the text of yt_res and renders it as an HTML document within our notebook!
HTML(yt_res.text)

### HTTP status codes

- When we **request** data from a website, the server includes an **HTTP status code** in the response.  

* The most common status code is `200`, which means there were no issues.  

* Other times, you will see a different status code, describing some sort of event or error.
    - Common examples: `403`: forbidden, `404`: page not found, `500`: internal server error.
    - [The first digit of a status describes its general "category."](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status)

- For example, the CSE faculty page doesn't let us scrape it.<br><small>Nothing is stopping us from opening Chrome, clicking "View Page Source", and manually downloading the HTML, though!</small>

In [ ]:
res = requests.get('https://cse.engin.umich.edu/people/faculty/') 
res.status_code

- As an aside, you can render HTML directly in a notebook using the `IPython.display.HTML` function.

In [ ]:
from IPython.display import HTML
HTML(res.text)

<div class="alert alert-danger" markdown="1">

#### Reference Slide

### Handling unsuccessful requests

- Sometimes, websites either don't want you to scrape, or prohibit you from scraping.<br><small>It's best practice to check the website's `robots.txt` file, where they specify who is and isn't allowed to scrape.<br>As we saw on the previous slide, the CSE website blocks us from scraping it, as we got a 403: Forbidden status code.</small>

- Some unsuccessful requests can be re-tried, depending on the issue.<br><small>A good first step is to wait a little, then try again.</small>

- A common issue is that you're making too many requests to a particular server at a time. If this is the case, you are being **rate-limited**; one solution is to increase the time between each request.<br><small>You can even do this programatically, say, using `time.sleep`.</small>

- See [LDS 14](https://learningds.org/ch/14/web_http.html) for more examples.

## The structure of HTML

---

### Scraping vs. APIs

- We mentioned earlier that there are two ways to programmatically access data on the internet: either **by scraping**, or **through an API**.

- **Scraping** is the act of emulating a web browser to access its HTML source code.<small>When scraping, you get back data as HTML and have to **parse** that HTML to extract the information you want. Parse means to "extract meaning from a sequence of symbols".

<center>
    
| ✅ Pros | ❌ Cons |
| --- | --- |
| If the website exists, you can usually scrape it.<br><small>This is what Google does!</small> | Scraping and parsing code gets **messy**, since <br>HTML documents contain lots of content unrelated to the<br>information you're trying to find (advertisements, formatting).<br><br>When the website's structure changes, your code will need to, too.<br><br>The site owner may not _want_ you to scrape it!</small>
    
    
</center>

- An application programming interface, or **API**, is a service that makes data directly available to the user in a **convenient** fashion. Usually, APIs give us code back as JSON objects.<br><small>APIs are made by organizations that host data. For example, X (formally known as Twitter) has an [API](https://developer.twitter.com/en/docs/twitter-api), as does [OpenAI](https://platform.openai.com/docs/overview?lang=python), the creators of ChatGPT.</small>


| ✅ Pros | ❌ Cons |
| --- | --- |
| If an API exists, the data are usually clean, up-to-date, and ready to use.<br><br>The presence of an API signals that the data provider<br> is okay with you using their data.<br><br>The data provider can plan and regulate data usage.<br><small>Sometimes, you may need to create an API "key",<br>which is like an account for using the API.<br>APIs can often give you access to data that isn't publicly available.</small> | APIs don't always exist for the data you want! |

- We're learning how to scrape now, and will see how to use APIs after.

### What is HTML?

- HTML (Hypertext Markup Language) is **the** basic building block of the internet. 

- It is a **markup language**, not a programming language.<br><small>Markup languages specify what something should _look like_, while programming languages specify what something should _calculate_ or _do_.</small>

- Specifically, it defines the content and layout of a webpage, and as such, it is what you get back when you scrape a webpage.

- We're only going to learn enough HTML to help us scrape information.<br><small>See [this tutorial](https://developer.mozilla.org/en-US/docs/Learn/Getting_started_with_the_web/HTML_basics) for more details on HTML.</small>

### An example webpage

- For instance, here's the source code of a very basic webpage.

In [ ]:
!cat data/lec09_ex1.html

- Here's what that webpage actually looks like:

In [ ]:
HTML('data/lec09_ex1.html')

### The anatomy of HTML documents

- **HTML document**: The totality of markup that makes up a webpage.

- **Document Object Model (DOM)**: The internal representation of an HTML document as a hierarchical **tree** structure.

- **HTML element**: An object in the DOM, such as a paragraph, header, or title.

- **HTML tags**: Markers that denote the **start** and **end** of an element, such as `<p>` and `</p>`.

- **Attributes**: Some tags have **attributes**, which further specify how to display information.

```html
        <p style="color: red">Look at my red text!</p>
```

### Example: Pages and trees

- For the following webpage:

<center>

<img src="imgs/webpage_anatomy.png" width=600>

</center>

- The DOM tree would look like:

<center><img src="imgs/dom_tree.png" width=600></center>

- **Why do we care?** Extracting information out of an HTML document will involve **traversing** this tree.

- See the attached reference slides for examples of common tags.

<div class="alert alert-danger" markdown="1">

#### Reference Slide

### Useful tags to know

- Often, the information we're looking for is nestled in one of these tags:

|Element|Description|
|:---|:---|
|`<html>`|the document|
|`<head>`|the header|
|`<body>`|the body|
|`<div>` |a logical division of the document|
|`<span>`|an *inline* logical division|
|`<p>`|a paragraph|
| `<a>`| an anchor (hyperlink)|
|`<h1>, <h2>, ...`| header(s) |
|`<img>`| an image |

- There are many, many more. See [this article](https://en.wikipedia.org/wiki/HTML_element) for examples.

<div class="alert alert-danger" markdown="1">

#### Reference Slide

### Example tags and attributes

- Tags can have **attributes**, which further specify how to display information on a webpage.

- For instance, `<img>` tags have `src` and `alt` attributes, among others:<br>

```html
        <img src="cool-visualization.png" alt="My box plot that I'm super proud of." width=500>
```

- Hyperlinks have `href` attributes: 

```html
        Click <a href="https://study.practicaldsc.org">this link</a> to access past exams.
```

- The `<div>` tag is one of the more common tags. It defines a "section" of an HTML document, and is often used as a container for other HTML elements.<br><small>Think of `<div>`s like cells in Jupyter Notebooks.</small>
    
```html
        <div class="background">
          <h3>This is a heading</h3>
          <p>This is a paragraph.</p>
        </div>
```

- Often, the information we're looking for is stored in an attribute!<br><small>You can imagine a situation where we want to get the URL behind a button, for example.</small>

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
<small>Remember that you can always ask questions anonymously at the link above!</small>
    
What lingering questions do we have about the Document Object Model and the structure of HTML?

## Parsing HTML

---

### Beautiful Soup 🍜

- [Beautiful Soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) is a Python HTML parser.<br><small>Remember, **parse** means to "extract meaning from a sequence of symbols".

- **Warning**: Beautiful Soup 4 and Beautiful Soup 3 work differently, so make sure you are using and looking at documentation for Beautiful Soup 4.<br><small>Rest assured, the `pds` conda environment already has Beautiful Soup 4 installed.</small>

### Example HTML document

- To start, we'll work with the source code for an HTML page with the DOM tree shown below:

<center><img src="imgs/dom_tree_1.png" width=600></center>

- The string `html_string` contains an HTML "document".

In [ ]:
html_string = '''
<html>
    <body>
      <div id="content">
        <h1>Heading here</h1>
        <p>My First paragraph</p>
        <p>My <em>second</em> paragraph</p>
        <hr>
      </div>
      <div id="nav">
        <ul>
          <li>item 1</li>
          <li>item 2</li>
          <li>item 3</li>
        </ul>
      </div>
    </body>
</html>
'''.strip()

In [ ]:
HTML(html_string)

### Instantiating `BeautifulSoup` objects

- `bs4`'s `BeautifulSoup` function takes in a string or file-like object representing HTML and returns a **parsed** document.

In [ ]:
# We also could have used:
# import bs4
# But, then we'd need to use bs4.BeautifulSoup every time.
from bs4 import BeautifulSoup

In [ ]:
BeautifulSoup?

- Normally, we pass the result of a `GET` request to `BeautifulSoup`, but here we will pass our hand-crafted `html_string`.

In [ ]:
soup = BeautifulSoup(html_string) 
soup

In [ ]:
type(soup)

- `BeautifulSoup` objects have several useful attributes, e.g. `text`:

In [ ]:
print(soup.text) 

### Finding elements in a BeautifulSoup object

- The two main methods you will use to extract information from a BeautifulSoup object are `find` and `find_all`.

- `soup.find(tag)` finds the **first** instance of a tag (the first one on the page, i.e. the first one that DFS sees), and returns just that tag.<br><small>It has several optional arguments: **look at the documentation!**</small>

- `soup.find_all(tag)` will find **all** instances of a tag, and returns a **list** of tags.

- Remember: **`find` finds tags!**

### Using `find`

- Let's try and extract the first `<div>` subtree.

<center><img src="imgs/dom_tree_1.png" width=600></center> 

In [ ]:
soup.find('div') 

<center><img src="imgs/dom_subtree_1.png" width=325></center>  

- Let's try and find the `<div>` element that has an `id` attribute equal to `'nav'`.

In [ ]:
soup.find('div', attrs={'id': 'nav'}) 

- `find` will return the first occurrence of a tag, regardless of its depth in the tree.

In [ ]:
# The ul child is not at the top of the tree, but we can still find it.
soup.find('ul') 

### Using `find_all`

- `find_all` returns a list of all matching tags.

In [ ]:
soup

In [ ]:
soup.find_all('div') 

In [ ]:
soup.find_all('li') 

- We often use the `find_all` method in conjunction with a `for`-loop or list comprehension, to perform some operation on every matching tag.

In [ ]:
[x.text for x in soup.find_all('li')] 

### Node attributes

- The `text` attribute of a tag element gets the text between the opening and closing tags.

In [ ]:
soup.find('p') 

In [ ]:
soup.find('p').text 

- The `attrs` attribute of a tag element lists all of its attributes.

In [ ]:
soup.find('div') 

In [ ]:
soup.find('div').text 

In [ ]:
soup.find('div').attrs 

- The `get` method of a tag element **gets the value of an attribute**.<br><small>`find` and `get` are easy to get confused, but you'll use them both a lot.</small>

In [ ]:
soup.find('div').get('id') 

- The `get` method must be called directly on the node that contains the attribute you're looking for.

In [ ]:
soup

In [ ]:
# While there are multiple 'id' attributes, none of them are in the <html> tag at the top.
soup.get('id') 

In [ ]:
soup.find('div').get('id') 

<div class="alert alert-success" markdown="1">
    <h3>Activity</h3>
    
Consider the following HTML document, which represents a webpage containing the top few songs with the most streams on Spotify today in Canada.

```html
<head>
    <title>3*Canada-2022-06-04</title>
</head>
<body>
    <h1>Spotify Top 3 - Canada</h1>
    <table>
        <tr class='heading'>
            <th>Rank</th>
            <th>Artist(s)</th> 
            <th>Song</th>
        </tr>
        <tr class=1>
            <td>1</td>
            <td>Harry Styles</td> 
            <td>As It Was</td>
        </tr>
        <tr class=2>
            <td>2</td>
            <td>Jack Harlow</td> 
            <td>First Class</td>
        </tr>
        <tr class=3>
            <td>3</td>
            <td>Kendrick Lamar</td> 
            <td>N95</td>
        </tr>
    </table>
</body>
```

- **Part 1**: How many leaf nodes are there in the DOM tree of the previous document — that is, how many nodes have no children?

- **Part 2**: What does the following line of code evaluate to?

```python
        len(soup.find_all("td"))
```

- **Part 3**: What does the following line of code evaluate to?

```python
        soup.find("tr").get("class")
```

## Example: Scraping quotes

---

### Example: Scraping quotes

- Navigate to [quotes.toscrape.com](https://quotes.toscrape.com).

<center><img src="imgs/quotes2scrape.png" width=60%></center>

- **Goal**: Extract quotes, and relevant metadata, into a DataFrame.

- Specifically, let's try to make a DataFrame that looks like the one below:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>quote</th>
      <th>author</th>
      <th>author_url</th>
      <th>tags</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</td>
      <td>Albert Einstein</td>
      <td>https://quotes.toscrape.com/author/Albert-Einstein</td>
      <td>change,deep-thoughts,thinking,world</td>
    </tr>
    <tr>
      <th>1</th>
      <td>“It is our choices, Harry, that show what we truly are, far more than our abilities.”</td>
      <td>J.K. Rowling</td>
      <td>https://quotes.toscrape.com/author/J-K-Rowling</td>
      <td>abilities,choices</td>
    </tr>
    <tr>
      <th>2</th>
      <td>“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”</td>
      <td>Albert Einstein</td>
      <td>https://quotes.toscrape.com/author/Albert-Einstein</td>
      <td>inspirational,life,live,miracle,miracles</td>
    </tr>
  </tbody>
</table>

### Organizing our work

- Eventually, we will implement a single function, `make_quote_df`, which takes in an integer `n` and returns a **DataFrame** with the quotes on the **first `n` pages** of [quotes.toscrape.com](https://quotes.toscrape.com).

- Along the way, we'll implement several helper functions, with the goal of separating our logic: **each function should either request information, OR parse, but not both!**

- This makes it easier to debug and catch errors.

- It also avoids **unnecessary requests**.

### Downloading a single page

- First, let's figure out how to download a single page from [quotes.toscrape.com](https://quotes.toscrape.com).

- The URLs seem to be formatted a very particular way:

```html
        https://quotes.toscrape.com/page/2
```

In [ ]:
def download_page(i):
    url = f'https://quotes.toscrape.com/page/{i}'
    res = requests.get(url)
    return BeautifulSoup(res.text)

- Let's test our function on a single page, like Page 2.<br><small>There's nothing special about Page 2; we chose it arbitrarily.</small>

In [ ]:
soup = download_page(2) 
soup

- Now that this works, later on, we can call `download_page(1)`, `download_page(2)`, `download_page(3)`, ..., `download_page(n)`.

### Parsing a single page

- Now, let's try and extract the relevant information out of the `soup` object for Page 2.

- **Open [quotes.toscrape.com/page/2](https://quotes.toscrape.com/page/2/) in Chrome, right click the page, and click "Inspect"!**<br><small>This will help us find where each quote is located in the HTML.</small>

In [ ]:
divs = soup.find_all('div', class_='quote') 
# The above is a shortcut for the following, just for when the attribute key is class:
# divs = soup.find_all('div', attrs={'class': 'quote'})

In [ ]:
divs[0]

- From this `<div>`, we can extract the quote, author name, author's URL, and tags.<br><small>Strategy: Figure out how to process one `<div>`, then put that logic in a function to use on other `<div>`s.

In [ ]:
# The quote.
divs[0].find('span', class_='text').text 

In [ ]:
# The author.
divs[0].find('small', class_='author').text 

In [ ]:
# The URL for the author.
divs[0].find('a').get('href') 

In [ ]:
# The quote's tags.
divs[0].find('meta', class_='keywords').get('content') 

### Parsing a single quote, and then a single page

- Let's implement a function that takes in a `<div>` corresponding to a single quote and returns a dictionary containing the quote's information.<br><small>Why use a dictionary? Passing `pd.DataFrame` a list of dictionaries is an easy way to create a DataFrame.</small>

In [ ]:
def process_quote(div):
    quote = div.find('span', class_='text').text
    author = div.find('small', class_='author').text
    author_url = 'https://quotes.toscrape.com' + div.find('a').get('href')
    tags = div.find('meta', class_='keywords').get('content')
    return {'quote': quote, 'author': author, 'author_url': author_url, 'tags': tags}

In [ ]:
# Make sure everything here looks correct based on what's on the webpage!
process_quote(divs[4]) 

- Now, we can implement a function that takes in a **list** of `<div>`s, calls `process_quote` on each `<div>` in the list, and returns a **DataFrame**.

In [ ]:
def process_page(divs):
    return pd.DataFrame([process_quote(div) for div in divs])

In [ ]:
process_page(divs)

### Putting it all together

- Now, we can implement `make_quote_df`.

In [ ]:
def make_quote_df(n):
    '''Returns a DataFrame containing the quotes on the first n pages of https://quotes.toscrape.com/.''' # This is called a docstring!
    dfs = []
    for i in range(1, n+1):
        # Download page n and create a BeautifulSoup object.
        soup = download_page(i)
        # Create DataFrame using the information in that page.
        divs = soup.find_all('div', class_='quote')
        df = process_page(divs)
        # Append DataFrame to dfs.
        dfs.append(df)
    # Stitch all DataFrames together.
    return pd.concat(dfs).reset_index(drop=True)

In [ ]:
quotes = make_quote_df(3)
quotes.head()

- Now, `quotes` is s DataFrame, like any other!

In [ ]:
quotes[quotes['author'] == 'Albert Einstein'] 

<div class="alert alert-danger" markdown="1">

#### Reference Slide

### Summary of our steps

1. Implement `download_page(i)`, which downloads a **single page** (page `i`) and returns a `BeautifulSoup` object of the response.

2. Implement `process_quote(div)`, which takes in a `<div>` tree corresponding to a **single quote** and returns a dictionary containing all of the relevant information for that quote.

3. Implement `process_page(divs)`, which takes in a list of `<div>` trees corresponding to a **single page** and returns a DataFrame containing all of the relevant information for all quotes on that page.

4. Implement `make_quote_df(n)`.

## Example: Scraping the Happening @ Michigan page

---

### Example: Scraping the Happening @ Michigan page

- As we stated earlier, our ultimate goal in today's lecture is to create a DataFrame with the information about each event at [events.umich.edu](https://events.umich.edu).

In [ ]:
res = requests.get('https://events.umich.edu')
res

In [ ]:
soup = BeautifulSoup(res.text)

- Let's start by opening the page in Chrome, right clicking on the page, and clicking "Inspect".<br><small>As we can see, the HTML is much more complicated this time – this is usually the case for real websites!</small>

### Identifying `<div>`s

- It's not easy identifying which `<div>`s we want. The Inspect tool makes this easier, but it's good to verify that `find_all` is finding the right number of elements.

In [ ]:
divs = soup.find_all(class_='col-xs-12') 

In [ ]:
len(divs)

- Again, let's deal with one `<div>` at a time. First, we should extract the title of the event.

In [ ]:
divs[0]

In [ ]:
divs[0].find('div', class_='event-title').find('a').get('title') 

- The time and location, too.

In [ ]:
divs[0].find('time').get('datetime') 

In [ ]:
divs[0].find('ul').find('a').get('title') 

### Parsing a single event, and then every event

- As before, we'll implement a function that takes in a BeautifulSoup object corresponding to a single `<div>` and returns a dictionary with the relevant information about that event.

In [ ]:
def process_event(div):
    title = div.find('div', class_='event-title').find('a').get('title')
    location = div.find('ul').find('a').get('title')
    time = pd.to_datetime(div.find('time').get('datetime')) # Good idea!
    return {'title': title, 'time': time, 'location': location}

In [ ]:
process_event(divs[12])

- Now, we can call it on every `<div>` in `divs`.<br><small>Remember, we already ran `divs = soup.find_all(class_='col-xs-12')`.</small>

In [ ]:
row_list = []
for div in divs:
    try:
        row_list.append(process_event(div))
    except Exception as e:
        print(e)

In [ ]:
events = pd.DataFrame(row_list) 
events.head()

- Now, `events` is a DataFrame, like any other!

In [ ]:
# Which events are in-person today?
events[~events['location'].isin(['Virtual', ''])] 

### Web data in practice

- [The spread of true and false news online](https://www.science.org/doi/full/10.1126/science.aap9559) by Vosoughi et al. compared how true and false news spreads via X (Twitter):

> There is worldwide concern over false news and the possibility that it can influence political, economic, and social well-being. To understand how false news spreads, Vosoughi et al. used a data set of rumor cascades on Twitter from 2006 to 2017. About 126,000 rumors were spread by ∼3 million people. False news reached more people than the truth; the top 1% of false news cascades diffused to between 1000 and 100,000 people, whereas the truth rarely diffused to more than 1000 people. Falsehood also diffused faster than the truth. The degree of novelty and the emotional reactions of recipients may be responsible for the differences observed.

- To conduct this study, the authors used the X API for accessing tweets and web-scraped fact-checking websites to verify whether news was false or not.

### Summary, next time

- Beautiful Soup is an HTML parser that allows us to (somewhat) easily extract information from HTML documents.
    - `soup.find` and `soup.find_all` are the methods you will use most often.

- When writing scraping code:
    - Use "inspect element" to identify the names of tags and attributes that are relevant to the information you want to extract.
    - Separate your logic for making requests and for parsing.

- **Next time**: Accessing data (and other services) through APIs.